In [1]:
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DEVICE

device(type='cuda')

### Подготовка данных

In [4]:
qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))

In [5]:
from collections import deque

questin_answer_data = []
for question_answer in qa_data:
    questin_answer_data.append(question_answer['question'])
    deque(map(questin_answer_data.append, question_answer['responses']))

In [6]:
questin_answer_data[:5]

['долго ли идут деньги с яндексденег на карту visa?',
 'нет. прорыв 35 ;)',
 'можно ли зарегистрировать авто в другом регионе',
 'можно на родственника из того региона.. .  а потом ездить по доверке',
 'что делать если у меня очень тонкие ногти а хочется их отрастить?']

In [7]:
with open('for_bpe.txt', 'w') as f:
    f.write('\n'.join(questin_answer_data))

In [8]:
del questin_answer_data

In [9]:
!head for_bpe.txt

долго ли идут деньги с яндексденег на карту visa?
нет. прорыв 35 ;)
можно ли зарегистрировать авто в другом регионе
можно на родственника из того региона.. .  а потом ездить по доверке
что делать если у меня очень тонкие ногти а хочется их отрастить?
витамины и умная эмаль (каждый день)
ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.
умная эмаль, витамины, йод, и поменьше крась лаком 
лаки фирмы trind производство usa + кальций
в чем отличие медитации от йоги?


In [10]:
VOCAB_SIZE = 30_000
MODEL_PATH = 'pretrained_bpe_lm.model'

In [11]:
yttm.BPE.train(data='for_bpe.txt', vocab_size=VOCAB_SIZE, model=MODEL_PATH)

In [12]:
tokenizer = yttm.BPE(model=MODEL_PATH)

In [13]:
questions = []
answers = []

for qa in qa_data:
    for answer in qa['responses']:
        questions.append(qa['question'])
        answers.append(answer)

In [14]:
del qa_data

In [15]:
batch_size = 256
tokenized_questions = []

for i_batch in tqdm(range(math.ceil(len(questions) / batch_size))):
    tokenized_questions.extend(
        tokenizer.encode(
            list(questions[i_batch*batch_size:(i_batch+1)*batch_size]),
            bos=True, eos=False,
        )
    )

100%|██████████| 30341/30341 [01:07<00:00, 447.11it/s]


In [16]:
# как сложно без gc
del questions

In [17]:
tokenized_answers = []

for i_batch in tqdm(range(math.ceil(len(answers) / batch_size))):
    tokenized_answers.extend(
        tokenizer.encode(
            list(answers[i_batch*batch_size:(i_batch+1)*batch_size]),
            bos=True, eos=False,
        )
    )

100%|██████████| 30341/30341 [01:07<00:00, 450.82it/s]


In [18]:
del answers
del tokenizer

In [20]:
# у меня не хватает памяти, лучше сохраниться
import pickle

with open('questions', 'wb') as f:
    pickle.dump(tokenized_questions, f)

with open('answers', 'wb') as f:
    pickle.dump(tokenized_answers, f)

In [21]:
del tokenized_questions
del tokenized_answers

### Датасет

In [30]:
# будем брать одну десятую датасета, иначе памяти не хватит
with open('questions', 'rb') as f:
    questions = pickle.load(f)
questions = questions[:int(len(questions)/10)]

In [32]:
with open('answers', 'rb') as f:
    answers = pickle.load(f)
answers = answers[:int(len(answers)/10)]

In [34]:
print(len(questions), len(answers))
assert len(questions) == len(answers)

776713 776713


In [37]:
# найс, у нас занято всего 3 Гб
!free -m

              total        used        free      shared  buff/cache   available
Mem:          14961        3574        8289         306        3097       10758
Swap:         16134        1232       14902


In [38]:
class SequenceBucketingData(torch.utils.data.Dataset):
    """по сути то же, что в условии, только другие сиквенсы"""
    def __init__(self, questions, answers, max_len, pad_index, eos_index):
        self.questions = questions
        self.answers = answers
        if len(questions) != len(answers):
            raise ValueError('Вопросы и ответы должны быть одной длины')
        self.max_len = max_len
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.questions)
    
    def prepare_sample(self, sequence_q, sequence_a, max_len):
        sequence_q = sequence_q[:max_len]
        sequence_a = sequence_a[:max_len]
        x = sequence_q
        y = secuence_a
        pads = [self.pad_index] * (max_len - len(x))
        x += pads
        y += pads
        return x, y
    
    def __getitem__(self, index):
        batch_q = self.questions[index]
        batch_a = self.answers[index]
        max_len = min([
            self.max_len,
            max(map(len, batch_q)),
            max(map(len, batch_a)),
        ])
        batch_x = []
        batch_y = []
        for sample_q, sample_a in zip(batch_q, batch_a):
            x, y = self.prepare_sample(sample_q, sample_a, max_len)
            batch_x.append(x)
            batch_y.append(y)
        batch_x = torch.tensor(batch_x).long().to(DEVICE)
        batch_y = torch.tensor(batch_y).long().to(DEVICE)
        return batch_x, batch_y